In [34]:
from tools.preprocess import *
from sklearn.neighbors import *
from sklearn.ensemble import *
from catboost import CatBoostRegressor
import catboost as cat
from xgboost import XGBRegressor, plot_importance
import xgboost as xgb
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold, cross_val_score
from tools.selector import *
from sklearn.metrics import r2_score
from clearml import Task
from rich.progress import *
from rich.console import Console
import time
pd.options.display.max_rows=400
pd.options.display.max_columns=400

import string 
import random
letters = string.digits
buzzwords = ["khrba","dar","hmza","mdrasa","ard"]
morewords = ["kbira","sghira","zwina","khayba","jdida","9dima"]
task_name = f'{random.choice(letters)}-{random.choice(buzzwords)}-{random.choice(morewords)}'

In [35]:
import yaml
from yaml.loader import SafeLoader

with open("preprocess.yaml","r") as f:
    preprocessing_parameters = yaml.load(f, Loader=SafeLoader)

Output()

In [36]:
X_train_0, Y_train_0, X_test_0, X_test_ids, X_train_ids = load_data('data/tabular/', False)

In [37]:

xgb_params, lgb_params, cat_params=load_hyperparameters()
X_train_1,Y_train_1, X_test_1 = preprocess(X_train_0, Y_train_0, X_test_0,preprocessing_parameters)


[20:40:25] Loading hyperparameters...                                                             ]8;id=475435;file:///home/alaassairi/classif/tools/preprocess.py\preprocess.py]8;;\:]8;id=666563;file:///home/alaassairi/classif/tools/preprocess.py#164\164]8;;\

           Fixing seeds                                                                            ]8;id=372528;file:///home/alaassairi/classif/tools/preprocess.py\preprocess.py]8;;\:]8;id=219684;file:///home/alaassairi/classif/tools/preprocess.py#65\65]8;;\

           Started preprocessing                                                                  ]8;id=288389;file:///home/alaassairi/classif/tools/preprocess.py\preprocess.py]8;;\:]8;id=256787;file:///home/alaassairi/classif/tools/preprocess.py#562\562]8;;\

           Adding geopopulation 2                                                                 ]8;id=709570;file:///home/alaassairi/classif/tools/preprocess.py\preprocess.py]8;;\:]8;id=776646;file:///home/alaassairi/classif/tools/preprocess.py#420\420]8;;\

           Frequency encoding ['city']                                                            ]8;id=442417;file:///home/alaassairi/classif/tools/preprocess.py\preprocess.py]8;;\:]8;id=33326;file:///home/alaassairi/classif/tools/preprocess.py#349\349]8;;\

...

[20:40:34] Adding extracted captions from images                                                  ]8;id=163032;file:///home/alaassairi/classif/tools/preprocess.py\preprocess.py]8;;\:]8;id=225772;file:///home/alaassairi/classif/tools/preprocess.py#710\710]8;;\

[20:40:35] Hot Encoding                                                                           ]8;id=398382;file:///home/alaassairi/classif/tools/preprocess.py\preprocess.py]8;;\:]8;id=101414;file:///home/alaassairi/classif/tools/preprocess.py#727\727]8;;\

           Appling robust scaling                                                                 ]8;id=277370;file:///home/alaassairi/classif/tools/preprocess.py\preprocess.py]8;;\:]8;id=846335;file:///home/alaassairi/classif/tools/preprocess.py#784\784]8;;\

           Dumping ['approximate_latitude']                                                       ]8;id=130889;file:///home/alaassairi/classif/tools/preprocess.py\preprocess.py]8;;\:]8;id=967096;file:///home/alaassairi/classif/tools/preprocess.py#801\801]8;;\

           Finished preprocessing                                                                 ]8;id=869693;file:///home/alaassairi/classif/tools/preprocess.py\preprocess.py]8;;\:]8;id=659176;file:///home/alaassairi/classif/tools/preprocess.py#808\808]8;;\

## Training & eval

In [7]:
# shitiest ids
# 36035051
# 35874833 
# 36014671 a pharmacy?
# 19187645 eeh
# cond = X_train_0["id_annonce"].isin(ids)
# X_train_1=X_train_1.loc[~cond]
# Y_train_1=Y_train_1.loc[~cond]



In [38]:

from sklearn.metrics import mean_absolute_error, mean_squared_error
xgb_params, lgb_params, cat_params=load_hyperparameters()
N_FOLD = 10
EARLY_STOP =250 

XGB_ITERATIONS = 100
LGB_ITERATIONS = 1000
CAT_ITERATIONS = 100


           Loading hyperparameters...                                                             ]8;id=605397;file:///home/alaassairi/classif/tools/preprocess.py\preprocess.py]8;;\:]8;id=201629;file:///home/alaassairi/classif/tools/preprocess.py#164\164]8;;\

In [39]:
def train_xgb(X_train, y_train, X_test):
    nfold = N_FOLD
    skf = KFold(n_splits=nfold, shuffle=True, random_state=2019)
    progress_bar = Progress(
        TextColumn('[progress.percentage]{task.percentage:>3.0f}%'),
        BarColumn(),
        MofNCompleteColumn(),
        TextColumn('•'),
        TimeElapsedColumn(),
        TextColumn('•'),
        TimeRemainingColumn(),
    )

    oof = np.zeros(len(X_train))
    predictions = np.zeros(len(X_train))
    final_predictions = np.zeros(len(X_test))

    predictors = X_train.columns.values.tolist()

    i = 1

    with progress_bar as progress:
        task1 = progress.add_task('[red]Training XGB', total=nfold)
        while not progress.finished:

            for train_index, valid_index in skf.split(X_train, y_train.values):
                # print("\nFold {}".format(i))
                xg_train = xgb.DMatrix(
                    X_train.iloc[train_index][predictors].values,
                    y_train.iloc[train_index].values,
                )
                xg_valid = xgb.DMatrix(
                    X_train.iloc[valid_index][predictors].values,
                    y_train.iloc[valid_index].values,
                )

                clf = xgb.train(xgb_params,
                                xg_train,
                                XGB_ITERATIONS,
                                evals=[(xg_train, 'train'),
                                       (xg_valid, 'eval')],
                                verbose_eval=False)
                oof[valid_index] = clf.predict(
                    xgb.DMatrix(X_train.iloc[valid_index][predictors].values))

                predictions += clf.predict(
                    xgb.DMatrix(X_train[predictors].values)) / nfold
                final_predictions += clf.predict(
                    (xgb.DMatrix(X_test[predictors].values))) / nfold

                i = i + 1
                progress.update(task1, advance=1)
                time.sleep(0.5)
    R2 = r2_score(y_train.values, oof)
    mae = mean_absolute_error(y_train.values, oof)
    mse = mean_squared_error(y_train.values, oof)
    rmse = np.sqrt(mse)
    console.log(
        f'R2 = {R2:<0.4f} --- MAE={mae:<0.4f} ---MSE={mse:<0.4f} --- RMSE ={rmse:<0.4f} | CV ={N_FOLD} ITERATIONS ={XGB_ITERATIONS}'
    )
    return final_predictions, predictions, R2, mae, mse, rmse


def train_lgb(X_train, y_train, X_test):
    nfold = N_FOLD
    skf = KFold(n_splits=nfold, shuffle=True, random_state=2019)
    progress_bar = Progress(
        TextColumn('[progress.percentage]{task.percentage:>3.0f}%'),
        BarColumn(),
        MofNCompleteColumn(),
        TextColumn('•'),
        TimeElapsedColumn(),
        TextColumn('•'),
        TimeRemainingColumn(),
    )
    oof = np.zeros(len(X_train))
    predictions = np.zeros(len(X_train))
    final_predictions = np.zeros(len(X_test))

    predictors = X_train.columns.values.tolist()

    i = 1
    with progress_bar as progress:
        task1 = progress.add_task('[red]Training LGB', total=nfold)
        while not progress.finished:
            for train_index, valid_index in skf.split(X_train, y_train.values):
                d_train = lgb.Dataset(
                    X_train.iloc[train_index][predictors].values,
                    y_train.iloc[train_index].values,
                )
                d_valid = lgb.Dataset(
                    X_train.iloc[valid_index][predictors].values,
                    y_train.iloc[valid_index].values,
                )
                watchlist = [d_valid]
                clf = lgb.train(lgb_params,
                                d_train,
                                num_boost_round=LGB_ITERATIONS,
                                valid_sets=d_valid,
                                callbacks=[lgb.log_evaluation(period=0)])
                oof[valid_index] = clf.predict(
                    X_train.iloc[valid_index][predictors].values)

                predictions += clf.predict(X_train[predictors].values) / nfold
                final_predictions += clf.predict(
                    X_test[predictors].values) / nfold
                i = i + 1
                progress.update(task1, advance=1)
                time.sleep(0.5)

    R2 = r2_score(y_train.values, oof)
    mae = mean_absolute_error(y_train.values, oof)
    mse = mean_squared_error(y_train.values, oof)
    rmse = np.sqrt(mse)
    console.log(
        f'R2 = {R2:<0.4f} --- MAE={mae:<0.4f} ---MSE={mse:<0.4f} --- RMSE ={rmse:<0.4f} | CV ={N_FOLD} ITERATIONS ={LGB_ITERATIONS}'
    )
    return final_predictions, predictions, R2, mae, mse, rmse


def train_cat(X_train, y_train, X_test):
    nfold = N_FOLD
    skf = KFold(n_splits=nfold, shuffle=True, random_state=2019)
    progress_bar = Progress(
        TextColumn('[progress.percentage]{task.percentage:>3.0f}%'),
        BarColumn(),
        MofNCompleteColumn(),
        TextColumn('•'),
        TimeElapsedColumn(),
        TextColumn('•'),
        TimeRemainingColumn(),
    )
    oof = np.zeros(len(X_train))
    predictions = np.zeros(len(X_train))
    final_predictions = np.zeros(len(X_test))
    predictors = X_train.columns.values.tolist()

    i = 1
    with progress_bar as progress:
        task1 = progress.add_task('[red]Training CAT', total=nfold)
        while not progress.finished:
            for train_index, valid_index in skf.split(X_train, y_train.values):
                # print("\nFold {}".format(i))
                d_train = cat.Pool(
                    X_train.iloc[train_index][predictors].values,
                    y_train.iloc[train_index].values,
                )
                d_valid = cat.Pool(
                    X_train.iloc[valid_index][predictors].values,
                    y_train.iloc[valid_index].values,
                )
                watchlist = [d_valid]
                clf = cat.train(d_train,
                                cat_params,
                                num_boost_round=CAT_ITERATIONS,
                                eval_set=d_valid,
                                early_stopping_rounds=EARLY_STOP,
                                verbose_eval=0)
                oof[valid_index] = clf.predict(
                    X_train.iloc[valid_index][predictors].values)

                predictions += clf.predict(X_train[predictors].values) / nfold
                final_predictions += clf.predict(
                    X_test[predictors].values) / nfold

                i = i + 1
                progress.update(task1, advance=1)
                time.sleep(0.5)
    R2 = r2_score(y_train.values, oof)
    mae = mean_absolute_error(y_train.values, oof)
    mse = mean_squared_error(y_train.values, oof)
    rmse = np.sqrt(mse)
    console.log(
        f'R2 = {R2:<0.4f} --- MAE={mae:<0.4f} ---MSE={mse:<0.4f} --- RMSE ={rmse:<0.4f} | CV ={N_FOLD} ITERATIONS ={CAT_ITERATIONS}'
    )
    return final_predictions, predictions, R2, mae, mse, rmse


In [40]:
xgb_preds,xgb_train_preds, r2_xgb, mae_xgb, mse_xgb, rmse_xgb = train_xgb(X_train_1, Y_train_1,X_test_1) 

# R2 = 0.8049 --- MAE=0.2548 ---MSE=0.1276 --- RMSE =0.3572 | CV =10 |  100 iterations (old hyper)
# R2 = 0.8096 --- MAE=0.2454 ---MSE=0.1246 --- RMSE =0.3529 | CV =10  (old hyper)
# R2 = 0.8113 --- MAE=0.2433 ---MSE=0.1234 --- RMSE =0.3513 | CV =25  (old hyper)

# R2 = 0.8184 --- MAE=0.2420 ---MSE=0.1188 --- RMSE =0.3447 | CV =10  | 100 iterations  (new hyper)
# R2 = 0.8192 --- MAE=0.2417 ---MSE=0.1183 --- RMSE =0.3439 | CV =10 ITERATIONS =100
# R2 = 0.8233 --- MAE=0.2363 ---MSE=0.1156 --- RMSE =0.3400 | CV =10 ITERATIONS =1000 (new hyper)
# R2 = 0.8257 --- MAE=0.2344 ---MSE=0.1140 --- RMSE =0.3377 | CV =25 ITERATIONS =1000 (new hyper)


Output()

[20:41:04] R2 = 0.8192 --- MAE=0.2417 ---MSE=0.1183 --- RMSE =0.3439 | CV =10 ITERATIONS =100      ]8;id=238968;file:///tmp/ipykernel_3657266/1199028262.py\1199028262.py]8;;\:]8;id=810620;file:///tmp/ipykernel_3657266/1199028262.py#58\58]8;;\

In [41]:
lgb_preds,lgb_train_preds, r2_lgb, mae_lgb, mse_lgb, rmse_lgb = train_lgb(X_train_1, Y_train_1,X_test_1)
# R2 = 0.8216 --- MAE=0.2328 ---MSE=0.1167 --- RMSE =0.3416 CV10 1000
# R2 = 0.8252 --- MAE=0.2310 ---MSE=0.1144 --- RMSE =0.3382 CV25 1000

# R2 = 0.8265 --- MAE=0.2283 ---MSE=0.1135 --- RMSE =0.3368 | CV =25 ITERATIONS 2000 (new)
# R2 = 0.8272 --- MAE=0.2274 ---MSE=0.1130 --- RMSE =0.3362 | CV =50 ITERATIONS 2000 (new)

Output()

[20:45:39] R2 = 0.8243 --- MAE=0.2301 ---MSE=0.1149 --- RMSE =0.3390 | CV =10 ITERATIONS =1000    ]8;id=908573;file:///tmp/ipykernel_3657266/1199028262.py\1199028262.py]8;;\:]8;id=105907;file:///tmp/ipykernel_3657266/1199028262.py#115\115]8;;\

In [19]:
cat_preds,cat_train_preds, r2_cat, mae_cat, mse_cat, rmse_cat = train_cat(X_train_1, Y_train_1,X_test_1)
# R2 = 0.8188 --- MAE=0.2386 ---MSE=0.1185 --- RMSE =0.3443 CV 10 

# R2 = 0.8230 --- MAE=0.2345 ---MSE=0.1158 --- RMSE =0.3402 | CV =25 ITERATIONS =3000    1199028262.py:173


Output()

[19:56:25] R2 = 0.8230 --- MAE=0.2345 ---MSE=0.1158 --- RMSE =0.3402 | CV =25 ITERATIONS =3000    ]8;id=958972;file:///tmp/ipykernel_3657266/1199028262.py\1199028262.py]8;;\:]8;id=674079;file:///tmp/ipykernel_3657266/1199028262.py#173\173]8;;\

## Inference


In [24]:
# weights = [5/40,15/40,20/40] 22,840577560550404

weights = [5/20,9/20, 6/20]  
# final_predictions_sum= (
#     weights[0]*np.exp(tuned_"models["xgboost"].predict(X_test_1)) +
#     weights[1]*np.exp(tuned_models["lgb"].predict(X_test_1)) + 
#     weights[2]*np.exp(tuned_models["cat"].predict(X_test_1)) 
#     )

final_predictions_sum= (
    weights[0]*np.exp(xgb_preds) +
    weights[1]*np.exp(lgb_preds) + 
    weights[2]*np.exp(cat_preds) 
    )

final_predictions=pd.Series(final_predictions_sum,name="price")
final_predictions.head()
final_submission = pd.concat([X_test_ids, final_predictions], axis=1)
final_submission["id_annonce"]=final_submission["id_annonce"].astype(np.int32)
final_submission.to_csv("data/final_submission_169.csv", index=False, header=True)
print("Finished submitting")


Finished submitting


In [15]:

train_sum= (
    weights[0]*np.exp(xgb_train_preds) +
    weights[1]*np.exp(lgb_train_preds) + 
    weights[2]*np.exp(cat_train_preds) 
    )

train_predictions=pd.Series(train_sum,name="price")
train_predictions.head()
train_submission = pd.concat([X_train_ids, train_predictions], axis=1)
train_submission["id_annonce"]=train_submission["id_annonce"].astype(np.int32)
train_submission.to_csv("data/train_submission.csv", index=False, header=True)
print("Finished submitting")

Finished submitting
